In [ ]:
# Set up the environment
# This cell only needed the first time you use this notebook on a system
import sys,os,os.path
os.environ['IBM_DB_HOME']='/home/ubuntu'

# Check to see if the libraries already have been installed
import importlib

# Check for ibm_db_sa.  If it exists, it's safe to assume that the other requirements
# are already installed.
spec = importlib.util.find_spec("ibm_db_sa")
if spec is None:
    print("Installing prerequisites.")
    !pip install ipython-sql
    !pip install "ibm-db==2.0.8a" 
    !pip install ibm_db_sa
else:
    print("sql magic, ibm_db and ibm_db_sa already installed.")
# Restart the Kernel if this is your first time installing the above. The next steps will fail unless you do this.
spec = importlib.util.find_spec("sqlparse")
if spec is None:
    print("Installing sqlparse.")
    !pip install sqlparse
else:
    print("sqlparse already installed.")


In [ ]:
# Import the modules and load the SQL magic
import ibm_db
import ibm_db_sa
import sqlalchemy
%load_ext sql
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
from IPython.display import display, HTML
import datetime
import time
from IPython.display import display, HTML, Markdown
import sqlparse
import getpass
pd.set_option('display.max_colwidth', -1)

In [ ]:
# Define filename for passwords
filename = 'ember_variables.py'
# source the file
%run $filename

# Report on Reocmmended Indexes

In [ ]:
print(datetime.datetime.now())

In [ ]:
#Connect to the database. Change the values of user, host, and password to match your environment. 
#For connection to a local host, use 'localhost' for the host name. 
#Also change the port number and database name.
user=''
host=''
password = getpass.getpass('Enter password for '+user)
inst=''
db=''
port=''

%sql db2+ibm_db://$user:$password@$host:$port/$db
            
print("Database: "+db)
print("Host: "+host)

In [ ]:
tabschema=''
explainschema=''
%sql set current schema :tabschema

In [ ]:
# Check to see if explain tables exist. Create them if they do not exist. Migrate them if they are invalid. Drop and recreate if that fails
ex_tabs=%sql call sysproc.sysinstallobjects('EXPLAIN','V',NULL,'SYSTOOLS')
if ex_tabs is None :
    display("Explain tables in the SYSTOOLS schema are invalid")
    %sql call sysproc.sysinstallobjects('EXPLAIN','M',NULL,'SYSTOOLS')
    %sql call sysproc.sysinstallobjects('EXPLAIN','C',NULL,'SYSTOOLS')
    ex_tabs=%sql call sysproc.sysinstallobjects('EXPLAIN','V',NULL,'SYSTOOLS')
    if ex_tabs is None :
        display('Migration failed, dropping and re-creating explain tables')
        %sql call sysproc.sysinstallobjects('EXPLAIN','D',NULL,'SYSTOOLS')
        %sql call sysproc.sysinstallobjects('EXPLAIN','C',NULL,'SYSTOOLS')
elif len(ex_tabs) == 0 : 
    display("Explain tables in the SYSTOOLS schema exist and are valid")
    
%sql update {explainschema}.advise_index set use_index='N' where exists='N'

In [ ]:
## Hide code cells
#HTML('''<script>
#code_show=true; 
#function code_toggle() {
# if (code_show){
# $('div.input').hide();
# } else {
# $('div.input').show();
# }
# code_show = !code_show
#} 
#$( document ).ready(code_toggle);
#</script>
#The raw code for this IPython notebook is by default hidden for easier reading.
#To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

## Statement
Define the statement this notebook focuses on. Only one statement per notebook

In [ ]:
dummy_val="50000"
dummy_date="2018-01-12"

In the box below between the first and last lines, you need to pasted in the single SQL statement this notebook focuses on. <br>
Any date parameter markers need to be replaced with ':dummy_date'. <br>
All other parameter markers will have dummy values filled in for you - just leave them as question marks.

In [ ]:
stmt="""\
SELECT ATTRVAL.ATTRVAL_ID,
       ATTRVAL.IDENTIFIER,
       ATTRVAL.ATTR_ID,
       ATTRVAL.FIELD1,
       ATTRVAL.FIELD2,
       ATTRVAL.FIELD3
FROM ATTRVAL
WHERE ATTRVAL.ATTR_ID IN (?,
                          ?,
                          ?,
                          ?)
  AND ATTRVAL.VALUSAGE is NOT NULL
  AND ATTRVAL.STOREENT_ID IN (?,
                              ?)
  AND ATTRVAL.ATTRVAL_ID IN
    (SELECT C1.ATTRVAL_ID
     FROM CATENTRYATTR C1,
          CATENTREL,
          CATENTRY
     WHERE C1.CATENTRY_ID = CATENTREL.CATENTRY_ID_CHILD
       AND CATENTREL.CATENTRY_ID_CHILD = CATENTRY.CATENTRY_ID
       AND CATENTRY.MARKFORDELETE = 0
       AND CATENTREL.CATENTRY_ID_PARENT IN (?) ) 
"""

In [ ]:
stmt=stmt.replace("?", ":dummy_val")
stmt=stmt.replace("{tabschema}", tabschema)

In [ ]:
# Get starting time
start_time=%sql select current timestamp from sysibm.sysdummy1
#start_time

In [ ]:
%sql set current explain mode explain
try:
    %sql $stmt
except Exception as e:
    error_string=str(e)
    if "CLI0115E" not in error_string:
        print(e)
## errors about an invalid cursor state can be ignored - that is what we expect here.

In [ ]:
time.sleep(10)

In [ ]:
%sql set current explain mode recommend indexes
try:
    %sql $stmt
except Exception as e: 
    error_string=str(e)
    if "CLI0115E" not in error_string:
        print(e)
## errors about an invalid cursor state can be ignored - that is what we expect here.

In [ ]:
%sql set current explain mode no

In [ ]:
time.sleep(10)

In [ ]:
# Get ending time
end_time=%sql select current timestamp from sysibm.sysdummy1
#end_time

In [ ]:
time.sleep(10)

### Reord Before and After Costs

In [ ]:
%sql set current schema :explainschema
expln_start_time=start_time[0][0].strftime('%Y-%m-%d-%H.%M.%S')
expln_end_time=end_time[0][0].strftime('%Y-%m-%d-%H.%M.%S')

In [ ]:
%%sql before_costs << select 
    dec(total_cost,20,4) as before_total_cost
    , dec(io_cost,20,4) as before_io_cost
    , dec(CPU_cost,20,4) as before_cpu_cost
    , dec(Comm_cost,20,4) as before_comm_cost
from Explain_Operator
    ,(select min(explain_time) as mintime
        from Explain_Operator 
        where operator_type = 'RETURN' 
          and explain_time between :expln_start_time and :expln_end_time) as b
where 
    explain_time = b.mintime
    and operator_type = 'RETURN' 
with UR ;

In [ ]:
display(before_costs)
before_costs_df=before_costs.DataFrame()
before_costs_df

In [ ]:
%%sql after_costs << select 
    dec(total_cost,20,4) as after_total_cost
    , dec(io_cost,20,4) as after_io_cost
    , dec(CPU_cost,20,4) as after_cpu_cost
    , dec(Comm_cost,20,4) as after_comm_cost
from Explain_Operator
    ,(select max(explain_time) as maxtime
        from Explain_Operator 
        where operator_type = 'RETURN' 
          and explain_time between :expln_start_time and :expln_end_time) as b
where 
    explain_time = b.maxtime
    and operator_type = 'RETURN' 
with UR ;

In [ ]:
after_costs_df=after_costs.DataFrame()
after_costs_df

In [ ]:
%%sql rec_ind_list << with ts as ( select max(explain_time) as maxtime
        from Explain_Operator 
        where operator_type = 'RETURN' 
          and explain_time between :expln_start_time and :expln_end_time)
select substr(name,1,25) as indname
    , substr(tbcreator,1,13) as tabschema
    , substr(tbname,1,18) as tabname
    , coalesce(si.fullkeycard, ai.fullkeycard) as fullkeycard
    , st.card
    , coalesce(si.uniquerule, ai.uniquerule) as uniquerule
    , use_index
    , exists
    , coalesce(si.colnames, ai.colnames) as colnames 
from advise_index ai 
    left outer join syscat.indexes si on ai.tbcreator=si.tabschema and ai.tbname=si.tabname and ai.name=si.indname 
    left outer join syscat.tables st on st.tabschema=ai.tbcreator and st.tabname=ai.tbname 
    join ts on explain_time = ts.maxtime
order by exists, use_index, uniquerule desc, indname with ur;

In [ ]:
#rec_ind_list

In [ ]:
rec_ind_df1=rec_ind_list.DataFrame()
#rec_ind_df=rec_ind_df.drop_duplicates
#rec_ind_df['card'] = rec_ind_df.apply(lambda x: "{:,}".format(x['card']), axis=1)
#rec_ind_df['fullkeycard'] = rec_ind_df.apply(lambda x: "{:,}".format(x['fullkeycard']), axis=1)
#rec_ind_df['colnames']=rec_ind_df['colnames'].replace('+','\n+',regex=False)
#display(HTML(rec_ind_df.to_html(index=False).replace("\\n","<br>")))
rec_ind_df=rec_ind_df1.drop_duplicates(subset=['indname'])
rec_ind_df.indname = rec_ind_df['indname'].map(lambda x: x.strip())
rec_ind_df.card = rec_ind_df.apply(lambda x: "{:,}".format(x['card']), axis=1)
rec_ind_df.fullkeycard = rec_ind_df.apply(lambda x: "{:,}".format(x['fullkeycard']), axis=1)
rec_ind_df.colnames=rec_ind_df.colnames.replace(to_replace='\+', value='\\n+', regex=True).replace(to_replace='-', value='\\n-', regex=True).str.lstrip('\n')
ind_list=rec_ind_df.indname.values.tolist()
ind_list_str=str(ind_list).strip('[]')

rec_ind_df=rec_ind_df.set_index('indname')
pd.set_option('display.max_colwidth', -1)
#display(HTML(rec_ind_df.to_html(index=True).replace("\\n","<br>")))

In [ ]:
this_cost_stmt="""\
select 
    dec(total_cost,20,4) as this_index_total_cost
    , dec(io_cost,20,4) as this_index_io_cost
    , dec(CPU_cost,20,4) as this_index_cpu_cost
    , dec(Comm_cost,20,4) as this_index_comm_cost
from Explain_Operator
    ,(select max(explain_time) as maxtime
        from Explain_Operator 
        where operator_type = 'RETURN') as b
where 
    explain_time = b.maxtime
    and operator_type = 'RETURN' 
with UR 
"""

In [ ]:
%sql update advise_index set use_index='N' where exists='N' and name in ({ind_list_str})
for index, row in rec_ind_df.iterrows():
    if row['EXISTS'] == 'N' :
        display(index)
        display(row)
        %sql update advise_index set use_index='Y' where name = :index
        %sql set current schema :tabschema
        %sql set current explain mode evaluate indexes
        try: 
            %sql $stmt
        except Exception as e: 
            error_string=str(e)
            if "CLI0115E" not in error_string:
                print(e)
        ## errors about an invalid cursor state can be ignored - that is what we expect here.
        %sql set current explain mode no
        %sql set current schema :explainschema
        display("this cost:")
        this_cost=%sql $this_cost_stmt
        this_cost_df=this_cost.DataFrame()
        display(this_cost)
        %sql update advise_index set use_index='N' where name = :index
        rec_ind_df.at[index, 'Index_Add']=before_costs_df.iloc[0]['before_total_cost']-this_cost_df.iloc[0]['this_index_total_cost']

In [ ]:
%sql update advise_index set use_index='Y' where name in ({ind_list_str})
for index, row in rec_ind_df.iterrows():
    if row['EXISTS'] == 'N' :
        display(index)
        display(row)
        %sql update advise_index set use_index='N' where name = :index
        %sql set current explain mode evaluate indexes
        %sql set current schema :tabschema
        try: 
            %sql $stmt
        except Exception as e: 
            error_string=str(e)
            if "CLI0115E" not in error_string:
                print(e)
        ## errors about an invalid cursor state can be ignored - that is what we expect here.
        %sql set current explain mode no
        %sql set current schema :explainschema
        display("this cost:")
        this_cost=%sql $this_cost_stmt
        this_cost_df=this_cost.DataFrame()
        display(this_cost)
        %sql update advise_index set use_index='Y' where name = :index
        rec_ind_df.at[index, 'Index_Subtr']=(before_costs_df.iloc[0]['before_total_cost']-after_costs_df.iloc[0]['after_total_cost'])-(before_costs_df.iloc[0]['before_total_cost']-this_cost_df.iloc[0]['this_index_total_cost'])

In [ ]:
display(HTML(rec_ind_df.to_html(index=True).replace("\\n","<br>")))

In [ ]:
display(before_costs_df)
display(after_costs_df)

In [ ]:
import ipywidgets as widgets
inds=widgets.SelectMultiple(
    options=rec_ind_df.query('EXISTS == "N"').index,
    value=list(rec_ind_df.query('EXISTS == "N" and Index_Add > 10000').index),
    #rows=10,
    description='New Indexes',
    disabled=False
)
display(inds)

In the box above, select the combination of indexes you would like to use and then re-run the cell below to try different index combinations.

In [ ]:
#inds_to_try_str=list(inds.value)
inds_to_try_str=str(list(inds.value)).strip("][\\")
inds_to_try_str

In [ ]:
#%sql update advise_index set use_index='N' where exists='N' and name in ({ind_list_str})
%sql update advise_index set use_index='N' where exists='N'
%sql update advise_index set use_index='Y' where name in ({inds_to_try_str})
%sql set current explain mode evaluate indexes
%sql set current schema :tabschema
try: 
    %sql $stmt
except Exception as e: 
    error_string=str(e)
    if "CLI0115E" not in error_string:
        print(e)
## errors about an invalid cursor state can be ignored - that is what we expect here.
%sql set current explain mode no
%sql set current schema :explainschema
display("this cost:")
this_cost=%sql $this_cost_stmt
this_cost_df=this_cost.DataFrame()
display(this_cost)
pct_savings=100*((before_costs_df.iloc[0]['before_total_cost']-this_cost_df.iloc[0]['this_index_total_cost'])/(before_costs_df.iloc[0]['before_total_cost']-after_costs_df.iloc[0]['after_total_cost']))
pct_imp=100*(this_cost_df.iloc[0]['this_index_total_cost']/before_costs_df.iloc[0]['before_total_cost'])
display("This combination of indexes will get {:.2f} percent of what adding all indexes would".format(pct_savings))
display("This combination of indexes represents a {:.2f} percent improvement over the current indexes".format(pct_imp))

In [ ]:
desc_tab_stmt = "select \
                    c.colname as column_name, \
                    c.colno as position, \
                    c.typename as data_type, \
                    c.length, \
                    c.scale, \
                    case when  c.nulls = 'Y' then 1 else 0 end as nullable, \
                    default as default_value, \
                    case when c.identity ='Y' then 1 else 0 end as is_identity, \
                    case when c.generated ='' then 0 else 1 end as  is_computed, \
                    c.text as computed_formula \
                from syscat.columns c \
                where c.tabschema=:tabschema and c.tabname=:tabname \
                order by position with ur "

In [ ]:
desc_ind_stmt = "select \
                    i.lastused \
                    , substr(indname,1,20) as indname \
                    , colnames \
                    , fullkeycard  as indcard \
                    , card as table_card \
                    , float(fullkeycard)/float(card) * 100 as card_pct \
                    , decimal(clusterfactor,10,5) as clusterfactor \
                    , indextype \
                from syscat.indexes i join syscat.tables t on i.tabname = t.tabname and i.tabschema = t.tabschema \
                where t.tabschema=:tabschema and t.tabname=:tabname order by float(fullkeycard)/float(card) * 100 desc with ur"

In [ ]:
#Index details
if inds.value:
    index_list = inds.value
else:
    index_list = rec_ind_df.query('EXISTS == "N"').index

for ind in index_list:
    display(Markdown("## Index "+str(ind)))
    display(Markdown("### Index Columns"))
    display(HTML(rec_ind_df['colnames'][ind]))
    ind_details=%sql select tbcreator, tbname, creation_text from {explainschema}.advise_index where name=:ind order by explain_time desc fetch first 1 rows only
    ind_det_df=ind_details.DataFrame()
    #display(ind_det_df)
    display(Markdown("### Index Creation Syntax"))
    display(ind_det_df['creation_text'][0])
    tabschema=ind_det_df['tbcreator'][0]
    tabname=ind_det_df['tbname'][0]
    display(Markdown("### Index is on table "+ind_det_df['tbcreator'][0]+"."+ind_det_df['tbname'][0]))
    display(Markdown('#### Table Structure'))
    tab_desc=%sql $desc_tab_stmt
    #display(tab_desc)
    tab_desc_df=tab_desc.DataFrame()
    tab_desc_df=tab_desc_df.set_index('POSITION')
    display(tab_desc_df)
    display(Markdown('#### Existing indexes on this table'))
    ind_desc=%sql $desc_ind_stmt
    ind_desc_df=ind_desc.DataFrame()
    ind_desc_df.colnames=ind_desc_df.colnames.replace(to_replace='\+', value='\\n+', regex=True).replace(to_replace='-', value='\\n-', regex=True).str.lstrip('\n')
    ind_desc_df=ind_desc_df.set_index('indname')
    display(HTML(ind_desc_df.to_html(index=True).replace("\\n","<br>")))
